## Feature Engineering

In [2]:
import pandas as pd
import featuretools as ft

In [42]:
df_anp = pd.read_pickle('../../data/Visual_well_being/anp.pickle')
df_celeb = pd.read_pickle('../../data/Visual_well_being/celebrity.pickle')
df_face = pd.read_pickle('../../data/Visual_well_being/face.pickle')
df_img_data = pd.read_pickle('../../data/Visual_well_being/image_data.pickle')
df_img_metrics = pd.read_pickle('../../data/Visual_well_being/image_metrics.pickle')
df_object = pd.read_pickle('../../data/Visual_well_being/object_labels.pickle')
df_survey = pd.read_pickle('../../data/Visual_well_being/survey.pickle')

In [4]:
import sys
lib_dir = "../../libs/"
if lib_dir not in sys.path:
    sys.path.append(lib_dir)

In [5]:
import image_level_agg_features as img_f
import user_level_agg_features as usr_f

In [6]:
# Create new entityset
es = ft.EntitySet(id = 'users')

In [8]:
df_survey.head()

,index,id,gender,born,education,employed,income,A_2,N_1,P_1,...,P,E,R,M,A,PERMA,N_EMO,P_EMO,imagecount,private_account
0,0,920bf027f7d13dbdc7b66b3d3324903c,Male,1975,College graduate,Employed for wages,"$30,000 to $39,999",4,5,5,...,4.333333,4.000000,4.666667,5.000000,3.666667,4.2500,4.000000,4.333333,465.0,public
1,1,b433b2bfe49e28d0b7c45925b53084e0,Male,1978,College graduate,Employed for wages,"$20,000 to $29,999",8,0,9,...,9.000000,7.333333,9.000000,8.000000,7.666667,8.2500,0.000000,9.000000,6.0,public
2,4,f4f54676f75f47c17dc434cf68845328,Female,1990,High school graduate,Employed for wages,"$80,000 to $89,999",7,3,8,...,8.000000,7.333333,7.666667,7.666667,7.333333,7.5625,3.333333,8.000000,767.0,public
3,5,a27a5fc47a59f35761705330253a58e3,Male,1997,High school graduate,Employed for wages,"$20,000 to $29,999",7,5,7,...,6.000000,4.666667,3.666667,6.666667,5.666667,5.3750,6.000000,6.000000,43.0,public
4,6,0a1002b2232a4ecbde604462f6d84bf9,Female,1993,College graduate,A student,"$20,000 to $29,999",7,5,7,...,5.666667,6.000000,6.000000,6.666667,7.000000,NaN,3.000000,5.666667,73.0,public


In [106]:
df_survey = df_survey.drop_duplicates('insta_user_id')

In [34]:
df_img_data.head()

,image_id,image_link,image_url,image_height,image_width,image_filter,image_posted_time_unix,image_posted_time,data_memorability,user_id,user_full_name,user_name,user_website,user_profile_pic,user_bio,user_followed_by,user_follows,user_posted_photos
0,1316962883971761394_3468175004,https://www.instagram.com/p/BJGysPxgsTy/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Lo-fi,1471214231,14-08-2016 22:37:11,0.800521,3468175004,Leah Jenkins,leah.chelle,,https://scontent.cdninstagram.com/t51.2885-19/...,,7.0,0.0,1.0
1,552382455733335946_263042348,https://www.instagram.com/p/eqdOq2JLeK/,https://scontent.cdninstagram.com/t51.2885-15/...,612.0,612.0,Normal,1380069141,25-09-2013 00:32:21,0.875568,263042348,Taylor Degruise,taylordegruise,,https://scontent.cdninstagram.com/t51.2885-19/...,,316.0,347.0,73.0
2,594552614686078174_263042348,https://www.instagram.com/p/hARnP2pLTe/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Vesper,1385096216,22-11-2013 04:56:56,0.672679,263042348,Taylor Degruise,taylordegruise,,https://scontent.cdninstagram.com/t51.2885-19/...,,316.0,347.0,73.0
3,553884883234370621_263042348,https://www.instagram.com/p/evy13fpLQ9/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Amaro,1380248245,27-09-2013 02:17:25,0.843525,263042348,Taylor Degruise,taylordegruise,,https://scontent.cdninstagram.com/t51.2885-19/...,,316.0,347.0,73.0
4,725551583154452417_263042348,https://www.instagram.com/p/oRrVIcJLfB/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Amaro,1400712510,21-05-2014 22:48:30,0.859796,263042348,Taylor Degruise,taylordegruise,,https://scontent.cdninstagram.com/t51.2885-19/...,,316.0,347.0,73.0


In [47]:
df_img_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41206 entries, 0 to 41205
Data columns (total 18 columns):
image_id                  41206 non-null object
image_link                41206 non-null object
image_url                 41206 non-null object
image_height              41206 non-null float64
image_width               41206 non-null float64
image_filter              41206 non-null object
image_posted_time_unix    41206 non-null object
image_posted_time         41206 non-null datetime64[ns]
data_memorability         41071 non-null float64
user_id                   41206 non-null object
user_full_name            41206 non-null object
user_name                 41206 non-null object
user_website              41206 non-null object
user_profile_pic          41206 non-null object
user_bio                  41206 non-null object
user_followed_by          41206 non-null float64
user_follows              41206 non-null float64
user_posted_photos        41206 non-null float64
dtypes: datet

In [45]:
df_img_data = pd.read_pickle('../../data/Visual_well_being/image_data.pickle')
df_img_data['image_posted_time'] = pd.to_datetime(df_img_data['image_posted_time'])

In [100]:
df_img_data['user_id'] = pd.to_numeric(df_img_data['user_id'])
df_survey['insta_user_id'] = pd.to_numeric(df_survey['insta_user_id'])

In [88]:
df_img_data.head(495).tail()

,image_id,image_link,image_url,image_height,image_width,image_filter,image_posted_time_unix,image_posted_time,data_memorability,user_id,user_full_name,user_name,user_website,user_profile_pic,user_bio,user_followed_by,user_follows,user_posted_photos
490,1008976405482769353_11520833,https://www.instagram.com/p/4AmrHrjQvJ/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Lark,1434499382,2015-06-17 00:03:02,0.785163,11520833,nᎥᔕᗩᔕᎥn,nisasin,,https://scontent.cdninstagram.com/t51.2885-19/...,"Go ahead, like my picture from 87 weeks ago. ...",201.0,299.0,312.0
491,994890984687477342_246535583,https://www.instagram.com/p/3OkBRUiSJe/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Normal,1432820269,2015-05-28 13:37:49,0.898208,246535583,sierra,ssjm1992,,https://instagram.fcgk8-1.fna.fbcdn.net/t51.28...,,165.0,85.0,3.0
492,334819379877847242_246535583,https://www.instagram.com/p/SlhHO5iSDK/,https://scontent.cdninstagram.com/t51.2885-15/...,612.0,612.0,Amaro,1354133602,2012-11-28 20:13:22,0.915828,246535583,sierra,ssjm1992,,https://instagram.fcgk8-1.fna.fbcdn.net/t51.28...,,165.0,85.0,3.0
493,994890234930471491_246535583,https://www.instagram.com/p/3Oj2XDiSJD/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Normal,1432820179,2015-05-28 13:36:19,0.896148,246535583,sierra,ssjm1992,,https://instagram.fcgk8-1.fna.fbcdn.net/t51.28...,,165.0,85.0,3.0
494,1084294418137953788_2143580844,https://www.instagram.com/p/8MMAdJDTX8/,https://scontent.cdninstagram.com/t51.2885-15/...,640.0,640.0,Normal,1443477988,2015-09-28 22:06:28,0.771214,2143580844,Jordan,cannabiscranium,http://Thirddegreedebirds.com,https://scontent.cdninstagram.com/t51.2885-19/...,Warp speed dont rainbow read me. 🐧💨,13.0,13.0,43.0


In [112]:
df_img_data_metrics = pd.merge(df_img_data, df_img_metrics, on='image_id')

In [114]:
es = es.entity_from_dataframe(entity_id = 'users', dataframe = df_survey, 
                              index = 'insta_user_id')

es = es.entity_from_dataframe(entity_id = 'image_data_metrics', dataframe = df_img_data_metrics, make_index=True, 
                              variable_types = {'user_bio': ft.variable_types.Text, 'user_website': ft.variable_types.Text}, index='data_metric_id', time_index='comment_count_time_created')

In [115]:
# Relationship between users and images
r_user_img_data_metrics = ft.Relationship(es['users']['insta_user_id'],
                                    es['image_data_metrics']['user_id'])


# Add the relationship to the entity set
es = es.add_relationship(r_user_img_data_metrics)

In [ ]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'users', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last', 'num_unique', 'min'],
                                 trans_primitives = ['years', 'month', 'subtract', 'divide'])